In [5]:
import torch
import torch.nn as nn
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, stacked_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.stack_layers = stacked_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers= stacked_layers, batch_first=True)
        self.inter = nn.Linear(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.stack_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.stack_layers, batch_size, self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [6]:
x_train = torch.load('test_tensor.pt')

In [8]:
from torchviz import make_dot
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files/Graphviz/bin'

input_size = 1  # Number of input features (Open, High, Low, Close)
hidden_size = 64  # Number of LSTM units
output_size = 1  # Number of output units (Close)
stacked_layers = 5

model = LSTMModel(input_size, hidden_size, output_size, stacked_layers)
model.load_state_dict(torch.load('modelo-MSFT.pt'))
# Assuming you have a model instance named 'model'
# Forward pass to generate the computational graph
y = model(x_train.to(device)).to('cpu')

# Visualize the computational graph
graph = make_dot(y, params=dict(model.named_parameters()))
graph.render('neural_network_graph', format='png')

'neural_network_graph.png'